<a href="https://colab.research.google.com/github/lyaminedb1/Restaurant-item-analyse-/blob/main/Mini_project_functionnal_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import jax
import jax.numpy as jnp
import optax
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("/content/diabetes_dataset.csv")

# Display basic information about the dataset
df_info = df.info()
df_head = df.head()

# Check for missing values
missing_values = df.isnull().sum()

# Display dataset overview
df_info, df_head, missing_values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9538 entries, 0 to 9537
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 9538 non-null   int64  
 1   Pregnancies         9538 non-null   int64  
 2   BMI                 9538 non-null   float64
 3   Glucose             9538 non-null   float64
 4   BloodPressure       9538 non-null   float64
 5   HbA1c               9538 non-null   float64
 6   LDL                 9538 non-null   float64
 7   HDL                 9538 non-null   float64
 8   Triglycerides       9538 non-null   float64
 9   WaistCircumference  9538 non-null   float64
 10  HipCircumference    9538 non-null   float64
 11  WHR                 9538 non-null   float64
 12  FamilyHistory       9538 non-null   int64  
 13  DietType            9538 non-null   int64  
 14  Hypertension        9538 non-null   int64  
 15  MedicationUse       9538 non-null   int64  
 16  Outcom

(None,
    Age  Pregnancies    BMI  Glucose  BloodPressure  HbA1c    LDL   HDL  \
 0   69            5  28.39    130.1           77.0    5.4  130.4  44.0   
 1   32            1  26.49    116.5           72.0    4.5   87.4  54.2   
 2   89           13  25.34    101.0           82.0    4.9  112.5  56.8   
 3   78           13  29.91    146.0          104.0    5.7   50.7  39.1   
 4   38            8  24.56    103.2           74.0    4.7  102.5  29.1   
 
    Triglycerides  WaistCircumference  HipCircumference   WHR  FamilyHistory  \
 0           50.0                90.5             107.9  0.84              0   
 1          129.9               113.3              81.4  1.39              0   
 2          177.6                84.7             107.2  0.79              0   
 3          117.0               108.9             110.0  0.99              0   
 4          145.9                84.1              92.8  0.91              0   
 
    DietType  Hypertension  MedicationUse  Outcome  
 0    

In [11]:
# Define features and target
features = ["Age", "BMI", "Glucose", "BloodPressure", "HbA1c", "LDL", "HDL", "Triglycerides", "WHR"]
target = "Outcome"  # Using Outcome as a proxy for fitness risk

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(df[features])
y = df[target].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to JAX tensors
X_train = jnp.array(X_train)
y_train = jnp.array(y_train)
X_test = jnp.array(X_test)
y_test = jnp.array(y_test)


# Define multiple basis functions
def polynomial_basis(x, degree=3):
    return jnp.concatenate([x**i for i in range(1, degree + 1)], axis=1)

def radial_basis(x, centers, gamma=1.0):
    # centers should have the same number of features as x
    centers = centers.reshape(1, -1, x.shape[1])  # Ensure proper broadcasting
    return jnp.exp(-gamma * jnp.linalg.norm(x[:, None, :] - centers, axis=-1) ** 2)

# Apply basis transformations
X_train_poly = polynomial_basis(X_train)
X_test_poly = polynomial_basis(X_test)

# Generate centers with the correct number of features
num_centers = 10
centers = jnp.linspace(-2, 2, num=num_centers).reshape(num_centers, 1)
# Tile centers to match the number of features
centers = jnp.tile(centers, (1, X_train.shape[1]))

X_train_rbf = radial_basis(X_train, centers)
X_test_rbf = radial_basis(X_test, centers)

# Initialize model parameters
key = jax.random.PRNGKey(0)
w = jax.random.normal(key, (X_train_poly.shape[1],))
b = jnp.array(0.0)

# Define function approximation model
def model(params, x):
    w, b = params
    return jnp.dot(x, w) + b

# Loss function (Mean Squared Error)
def loss_fn(params, x, y):
    predictions = model(params, x)
    return jnp.mean((predictions - y) ** 2)

# Define training step
def train_step(params, opt_state, x, y, optimizer):
    loss, grads = jax.value_and_grad(loss_fn)(params, x, y)
    updates, opt_state = optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss

# Compare multiple optimization algorithms with learning rate scheduling
schedule = optax.exponential_decay(init_value=0.01, transition_steps=100, decay_rate=0.95)
grad_clip = 1.0  # Gradient clipping threshold

optimizers = {
    "Adam": optax.chain(optax.clip_by_global_norm(grad_clip), optax.adam(schedule)),
    "SGD": optax.chain(optax.clip_by_global_norm(grad_clip), optax.sgd(schedule)),
    "RMSProp": optax.chain(optax.clip_by_global_norm(grad_clip), optax.rmsprop(schedule))
}

# Train and evaluate different optimizers
results = {}
for name, optimizer in optimizers.items():
    opt_state = optimizer.init((w, b))
    params = (w, b)

    print(f"\nTraining with {name}...")
    for epoch in range(500):
        params, opt_state, loss = train_step(params, opt_state, X_train_poly, y_train, optimizer)
        if epoch % 50 == 0:
            print(f"Optimizer: {name}, Epoch {epoch}, Loss: {loss:.4f}")

    # Test performance
    y_pred = model(params, X_test_poly)
    mse = jnp.mean((y_pred - y_test) ** 2)
    results[name] = mse
    print(f"Final Test MSE with {name}: {mse:.4f}")

# Print results
print("\nOptimizer Comparison:")
for opt, mse in results.items():
    print(f"{opt}: MSE = {mse:.4f}")



Training with Adam...
Optimizer: Adam, Epoch 0, Loss: 238.3786
Optimizer: Adam, Epoch 50, Loss: 65.5503
Optimizer: Adam, Epoch 100, Loss: 11.0547
Optimizer: Adam, Epoch 150, Loss: 1.1261
Optimizer: Adam, Epoch 200, Loss: 0.2630
Optimizer: Adam, Epoch 250, Loss: 0.2155
Optimizer: Adam, Epoch 300, Loss: 0.2095
Optimizer: Adam, Epoch 350, Loss: 0.2087
Optimizer: Adam, Epoch 400, Loss: 0.2087
Optimizer: Adam, Epoch 450, Loss: 0.2087
Final Test MSE with Adam: 0.2106

Training with SGD...
Optimizer: SGD, Epoch 0, Loss: 238.3786
Optimizer: SGD, Epoch 50, Loss: 169.8994
Optimizer: SGD, Epoch 100, Loss: 116.2994
Optimizer: SGD, Epoch 150, Loss: 75.9564
Optimizer: SGD, Epoch 200, Loss: 47.1178
Optimizer: SGD, Epoch 250, Loss: 27.8169
Optimizer: SGD, Epoch 300, Loss: 15.8845
Optimizer: SGD, Epoch 350, Loss: 9.2381
Optimizer: SGD, Epoch 400, Loss: 6.0302
Optimizer: SGD, Epoch 450, Loss: 4.2906
Final Test MSE with SGD: 3.1091

Training with RMSProp...
Optimizer: RMSProp, Epoch 0, Loss: 238.3786
Op